In [1]:
import random
import numpy as np
import tensorflow as tf

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from sktime.datasets import load_UCR_UEA_dataset

from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import (Flatten, Dense, Dropout, 
BatchNormalization, Input)

In [2]:
seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
X_train, y_train = load_UCR_UEA_dataset(name="Strawberry",
                                        split='train', 
                                        return_type='numpy3D',
                                        extract_path="./data")
X_test, y_test = load_UCR_UEA_dataset(name="Strawberry",
                                      split='test', 
                                      return_type='numpy3D',
                                      extract_path='./data')

In [4]:
def prepare_labels(y):
    unique_classes = np.unique(y)
    class_mapping = {label: idx for idx, label in enumerate(unique_classes)}
    y_int = np.array([class_mapping[label] for label in y])
    y_cat = to_categorical(y_int)
    
    return y_cat

In [5]:
def create_model(
    input_shape, 
    num_classes
):
    model = Sequential([
        # Input layer
        Input(shape=input_shape),
        
        # Flatten the time series input
        Flatten(),
        
        # Dense layers
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [6]:
def train_model(
    model, 
    X_train, 
    y_train, 
    validation_split=0.2,
    batch_size=32, 
    epochs=100, 
    patience=3
):
    X_train_final, X_val, y_train_final, y_val = train_test_split(
        X_train, y_train,
        test_size=validation_split,
        random_state=42
    )
    
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=patience,
        restore_best_weights=True
    )
    
    history = model.fit(
        X_train_final, y_train_final,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping]
    )
    
    return history

In [7]:
def predict(
    model, 
    X, 
    return_probabilities=False
):
    y_pred_proba = model.predict(X)
    
    if return_probabilities:
        return y_pred_proba
    else:
        return np.argmax(y_pred_proba, axis=1)

In [8]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

# Shift values to start from 0 if needed (only if your labels are 1,2 instead of 0,1)
# Otherwise, Keras makes three labels that start at 0, but this label is never used
y_train = y_train - 1
y_test = y_test - 1
y_train_cat = prepare_labels(y_train)
y_test_cat = prepare_labels(y_test)

In [9]:
n_features = X_train.shape[1]
n_timesteps = X_train.shape[2]
n_classes = 2
    
model = create_model(
    input_shape=(n_features, n_timesteps), 
    num_classes=n_classes
)

In [10]:
history = train_model(model, X_train, y_train_cat)

Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.5789 - loss: 1.0922 - val_accuracy: 0.8130 - val_loss: 0.6234
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7125 - loss: 0.6688 - val_accuracy: 0.5610 - val_loss: 0.7005
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8403 - loss: 0.4160 - val_accuracy: 0.4146 - val_loss: 0.8241
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8151 - loss: 0.4165 - val_accuracy: 0.3984 - val_loss: 1.0283


In [11]:
y_pred = predict(model, X_test)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


In [12]:
clf_report = classification_report(y_test, y_pred)

print(clf_report)

              precision    recall  f1-score   support

           0       0.82      0.70      0.76       132
           1       0.85      0.92      0.88       238

    accuracy                           0.84       370
   macro avg       0.84      0.81      0.82       370
weighted avg       0.84      0.84      0.84       370

